In [2]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
import math
import json
from pandas.io.json import json_normalize

In [3]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

pensActivity = pd.read_sql('SELECT * FROM PensActivity', con=db_connection)
pensActivity['created'] = pd.to_datetime(pensActivity['created'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
pensActivity['lastmodified'] = pd.to_datetime(pensActivity['lastmodified'], unit='s').dt.tz_localize('utc').dt.tz_convert('US/Eastern')
pensActivity['created'].replace(0, np.nan, inplace=True)
pensActivity['lastmodified'].replace(0, np.nan, inplace=True)
pensActivity['notes'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
pensActivity['notes'] = pensActivity['notes'].astype('category')
pensActivity.sort_values(by='created', inplace=True)

In [3]:
pensActivity_openinghours = pensActivity.set_index('created')
pensActivity_openinghours = pensActivity_openinghours.between_time('10:00', '18:00')
pensActivity_openinghours.reset_index(inplace=True)

## total collected items

In [4]:
def total_collected_items():
    return pensActivity_openinghours['count_items'].sum()

In [5]:
total_collected_items()

19486350

## collected items per month

In [9]:
def collected_items_per_month():
    items_by_date = pensActivity[['count_items', 'created']]
    per_month = items_by_date.created.dt.to_period("M")
    month_count = items_by_date.groupby(per_month).sum()
    return month_count

In [10]:
collected_items_per_month()

,count_items
created,
2014-12,626
2015-01,163
2015-02,30565
2015-03,404963
2015-04,450773
...,...
2020-02,95136
2020-03,12551
2020-04,10


## activity column 

In [4]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 50)
pd.set_option("max_colwidth", 1000)

In [5]:
activity_parse = pensActivity['activity']

def parse(string):
    if string is None:
        return {}
    else:
        try:
            return json.loads(string)
        except ValueError:
            print(string)
            return {}
        
    
json_str = activity_parse.map(parse)

{"collection":[{"record_data":"chsdm:o:18487435","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:18638637","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:18615569","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:18353745","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:18562521","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:35460793","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:18400931","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:18471355","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:18667891","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:18758367","record_id":1,"record_len":"16","timestamp":"18:34:46"},{"record_data":"chsdm:o:18471361","record_id":1,"record_len":"16","timestamp":"18:34:

{"collection":[{"record_data":"chsdm:v:69166469","record_id":0,"record_len":"16","timestamp":"00:48:10"},{"record_data":"chsdm:o:18758389","record_id":1,"record_len":"16","timestamp":"00:48:26"},{"record_data":"chsdm:o:18733333","record_id":2,"record_len":"16","timestamp":"00:48:42"},{"record_data":"chsdm:o:69153301","record_id":3,"record_len":"16","timestamp":"00:48:48"},{"record_data":"chsdm:o:18631613","record_id":4,"record_len":"16","timestamp":"00:48:51"},{"record_data":"chsdm:o:18630461","record_id":5,"record_len":"16","timestamp":"00:48:53"},{"record_data":"chsdm:o:18630459","record_id":6,"record_len":"16","timestamp":"00:48:53"},{"record_data":"chsdm:o:18630457","record_id":7,"record_len":"16","timestamp":"00:48:53"},{"record_data":"chsdm:o:18630455","record_id":8,"record_len":"16","timestamp":"00:48:53"},{"record_data":"chsdm:o:69153299","record_id":9,"record_len":"16","timestamp":"00:48:56"},{"record_data":"chsdm:o:69153301","record_id":10,"record_len":"16","timestamp":"00:48

{"collection":[{"record_data":"chsdm:o:69155335","record_id":0,"record_len":"16","timestamp":"00:00:32"},{"record_data":"chsdm:o:69155057","record_id":1,"record_len":"16","timestamp":"00:01:31"},{"record_data":"chsdm:o:69155275","record_id":2,"record_len":"16","timestamp":"00:01:49"},{"record_data":"chsdm:o:69155275","record_id":3,"record_len":"16","timestamp":"00:02:08"},{"record_data":"chsdm:o:69155331","record_id":4,"record_len":"16","timestamp":"00:03:50"},{"record_data":"chsdm:o:69155065","record_id":5,"record_len":"16","timestamp":"00:04:03"},{"record_data":"chsdm:o:69155063","record_id":6,"record_len":"16","timestamp":"00:04:03"},{"record_data":"chsdm:o:69155003","record_id":7,"record_len":"16","timestamp":"00:04:03"},{"record_data":"chsdm:o:69155333","record_id":8,"record_len":"16","timestamp":"00:04:29"},{"record_data":"chsdm:o:69155339","record_id":9,"record_len":"16","timestamp":"00:04:52"},{"record_data":"chsdm:o:69155337","record_id":10,"record_len":"16","timestamp":"00:04

{"collection":[{"record_data":"chsdm:o:69155335","record_id":0,"record_len":"16","timestamp":"00:01:34"},{"record_data":"chsdm:o:69155335","record_id":1,"record_len":"16","timestamp":"00:01:38"},{"record_data":"chsdm:o:18383473","record_id":2,"record_len":"16","timestamp":"00:06:18"},{"record_data":"chsdm:o:18489265","record_id":3,"record_len":"16","timestamp":"00:06:20"},{"record_data":"chsdm:o:18659073","record_id":4,"record_len":"16","timestamp":"00:06:22"},{"record_data":"chsdm:o:18184945","record_id":5,"record_len":"16","timestamp":"00:06:23"},{"record_data":"chsdm:o:18468071","record_id":6,"record_len":"16","timestamp":"00:06:25"},{"record_data":"chsdm:o:18135781","record_id":7,"record_len":"16","timestamp":"00:06:27"},{"record_data":"chsdm:o:18327301","record_id":8,"record_len":"16","timestamp":"00:06:29"},{"record_data":"chsdm:o:18168763","record_id":9,"record_len":"16","timestamp":"00:06:32"},{"record_data":"chsdm:o:18489265","record_id":10,"record_len":"16","timestamp":"00:06

{"collection":[{"record_data":"chsdm:o:219777433","record_id":0,"record_len":"17","timestamp":"00:50:51"},{"record_data":"chsdm:o:136252991","record_id":1,"record_len":"17","timestamp":"00:50:51"},{"record_data":"chsdm:o:219777431","record_id":2,"record_len":"17","timestamp":"00:50:51"},{"record_data":"chsdm:o:219777427","record_id":3,"record_len":"17","timestamp":"00:50:51"},{"record_data":"chsdm:o:18624741","record_id":4,"record_len":"16","timestamp":"00:51:38"},{"record_data":"chsdm:o:18636273","record_id":5,"record_len":"16","timestamp":"00:51:40"},{"record_data":"chsdm:o:18468069","record_id":6,"record_len":"16","timestamp":"00:51:42"},{"record_data":"chsdm:o:18670471","record_id":7,"record_len":"16","timestamp":"00:51:53"},{"record_data":"chsdm:o:18732761","record_id":8,"record_len":"16","timestamp":"00:51:54"},{"record_data":"chsdm:o:18636375","record_id":9,"record_len":"16","timestamp":"00:51:56"},{"record_data":"chsdm:o:136253113","record_id":10,"record_len":"17","timestamp":"

...

{"collection":[{"record_data":"chsdm:o:69155013","record_id":0,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:69155009","record_id":1,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:69155007","record_id":2,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:69154997","record_id":3,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:69154999","record_id":4,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:69155005","record_id":5,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:69155017","record_id":6,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:69155015","record_id":7,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:69155011","record_id":8,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:69155057","record_id":9,"record_len":"16","timestamp":"06:16:14"},{"record_data":"chsdm:o:102335191","record_id":10,"record_len":"17","timestamp":"06:1

{"collection":[{"record_data":"chsdm:o:907131605","record_id":0,"record_len":"17","timestamp":"00:57:43"},{"record_data":"chsdm:o:907214293","record_id":1,"record_len":"17","timestamp":"00:57:47"},{"record_data":"chsdm:o:18343959","record_id":2,"record_len":"16","timestamp":"00:57:49"},{"record_data":"chsdm:o:907130359","record_id":3,"record_len":"17","timestamp":"00:57:51"},{"record_data":"chsdm:o:404536641","record_id":4,"record_len":"17","timestamp":"00:57:55"},{"record_data":"chsdm:o:18353701","record_id":5,"record_len":"16","timestamp":"00:57:57"},{"record_data":"chsdm:o:908038735","record_id":6,"record_len":"17","timestamp":"00:58:03"},{"record_data":"chsdm:o:907130253","record_id":7,"record_len":"17","timestamp":"00:58:06"},{"record_data":"chsdm:o:35521009","record_id":8,"record_len":"16","timestamp":"00:58:08"},{"record_data":"chsdm:o:404536663","record_id":9,"record_len":"17","timestamp":"00:58:11"},{"record_data":"chsdm:o:18794697","record_id":10,"record_len":"16","timestamp"

{"collection":[{"record_data":"chsdm:o:18187831","record_id":0,"record_len":"16","timestamp":"00:01:50"},{"record_data":"chsdm:o:907130101","record_id":1,"record_len":"17","timestamp":"00:07:03"},{"record_data":"chsdm:o:18312593","record_id":2,"record_len":"16","timestamp":"00:07:08"},{"record_data":"chsdm:o:18574871","record_id":3,"record_len":"16","timestamp":"00:07:12"},{"record_data":"chsdm:o:18574871","record_id":4,"record_len":"16","timestamp":"00:07:55"},{"record_data":"chsdm:o:907214171","record_id":5,"record_len":"17","timestamp":"00:08:04"},{"record_data":"chsdm:o:907214199","record_id":6,"record_len":"17","timestamp":"00:08:06"},{"record_data":"chsdm:o:907130393","record_id":7,"record_len":"17","timestamp":"00:08:18"},{"record_data":"chsdm:o:907130427","record_id":8,"record_len":"17","timestamp":"00:08:20"},{"record_data":"chsdm:o:907214249","record_id":9,"record_len":"17","timestamp":"00:08:23"},{"record_data":"chsdm:o:907214133","record_id":10,"record_len":"17","timestamp"

{"collection":[{"record_data":"chsdm:o:1108723777","record_id":0,"record_len":"18","timestamp":"00:03:09"},{"record_data":"chsdm:o:68813711","record_id":1,"record_len":"16","timestamp":"00:03:09"},{"record_data":"chsdm:o:1108723777","record_id":2,"record_len":"18","timestamp":"00:03:18"},{"record_data":"chsdm:o:68813711","record_id":3,"record_len":"16","timestamp":"00:03:18"},{"record_data":"chsdm:o:1108723777","record_id":4,"record_len":"18","timestamp":"00:03:55"},{"record_data":"chsdm:o:68813711","record_id":5,"record_len":"16","timestamp":"00:03:55"},{"record_data":"chsdm:o:1108723781","record_id":6,"record_len":"18","timestamp":"00:04:17"},{"record_data":"chsdm:o:404577379","record_id":7,"record_len":"17","timestamp":"00:04:17"},{"record_data":"chsdm:o:907130463","record_id":8,"record_len":"17","timestamp":"00:04:25"},{"record_data":"chsdm:o:1108723791","record_id":9,"record_len":"18","timestamp":"00:04:25"},{"record_data":"chsdm:o:890290247","record_id":10,"record_len":"17","time

{"collection":[{"record_id":1,"record_data":"chsdm:o:1108750399","record_length":18,"timestamp":"01:26:01"},{"record_id":2,"record_data":"chsdm:o:18651137","record_length":16,"timestamp":"01:26:41"},{"record_id":3,"record_data":"chsdm:o:18651119","record_length":16,"timestamp":"01:26:43"},{"record_id":4,"record_data":"chsdm:o:18669115","record_length":16,"timestamp":"01:26:46"},{"record_id":5,"record_data":"chsdm:o:18768149","record_length":16,"timestamp":"01:26:48"},{"record_id":6,"record_data":"chsdm:o:18768143","record_length":16,"timestamp":"01:26:48"},{"record_id":7,"record_data":"chsdm:o:18495073","record_length":16,"timestamp":"01:26:57"},{"record_id":8,"record_data":"chsdm:o:18319945","record_length":16,"timestamp":"01:26:59"},{"record_id":9,"record_data":"chsdm:o:1108754281","record_length":18,"timestamp":"01:27:01"},{"record_id":10,"record_data":"chsdm:o:69122329","record_length":16,"timestamp":"01:27:06"},{"record_id":11,"record_data":"chsdm:o:18497979","record_length":16,"t

In [6]:
json_str.map(lambda row: row.get('collection'))
json_list = list(json_str)
activity_df = pd.json_normalize(json_list)
collection_json = activity_df['collection']

def record_map(record):
    try:
        return list(map(lambda data: data['record_data'], record))
    except TypeError:
        return None
    
activity_df['record_data'] = activity_df['collection'].apply(record_map)

In [7]:
activity_df

,collection,ticket_id,pen_id,initial_time_stamp,registers.vibra_on,registers.tag_counter_after_low_battery_level_reached,registers.p2p_life_counter,registers.tag_life_counter,registers.pen_id,registers.vibra_duration,registers.company,registers.tag_counter_before_low_battery_level_reached,registers.nfc_loops,registers.p2p_test_normal_mode,registers.low_battery_level_reached_since_last_reset,registers.p2p_counter_after_low_battery_level_reached,registers.p2p_counter_before_low_battery_level_reached,pen_uid,unit_id,registers,initial_timestamp,registers.pen_uid,registers.unit_id,record_data
0,"[{'record_id': 1, 'record_data': 'chsdm:o:18500983', 'record_len': 16, 'timestamp': '00:00:04'}, {'record_id': 1, 'record_data': 'chsdm:o:18305299', 'record_len': 16, 'timestamp': '00:00:07'}]",20141215u3,5,2014-12-15T15:33:19,1.0,0.0,378.0,29.0,NaN,115973.0,0x0,29.0,16.0,1.0,0.0,0.0,378.0,NaN,NaN,NaN,NaN,NaN,NaN,"[chsdm:o:18500983, chsdm:o:18305299]"
1,"[{'record_id': 1, 'record_data': 'Uchsdm:o:18797297;18796201;35460625;35474659;68245681;18740273;51685081;68268445;52027703;18797267;68268299', 'record_len': 110, 'timestamp': '0:35:54'}, {'record_id': 2, 'record_data': 'Uchsdm:o:18732295', 'record_len': 20, 'timestamp': '0:36:21'}, {'record_id': 3, 'record_data': 'Uchsdm:o:51497725;51497729;51497727', 'record_len': 38, 'timestamp': '0:37:1'}, {'record_id': 4, 'record_data': 'Uchsdm:o:51497641', 'record_len': 20, 'timestamp': '0:37:4'}, {'record_id': 5, 'record_data': 'Uchsdm:o:18319405', 'record_len': 20, 'timestamp': '0:38:42'}, {'record_id': 6, 'record_data': 'Uchsdm:o:18708929', 'record_len': 20, 'timestamp': '0:39:13'}, {'record_id': 7, 'record_data': 'Uchsdm:o:51681337', 'record_len': 20, 'timestamp': '0:39:27'}]",sx6w,NaN,#2014-12-15T15:06:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91ff826f0b001e00,ffff,,NaN,NaN,NaN,"[Uchsdm:o:18797297;18796201;35460625;35474659;68245681;18740273;51685081;68268445;52027703;18797267;68268299, Uchsdm:o:18732295, Uchsdm:o:51497725;51497729;51497727, Uchsdm:o:51497641, Uchsdm:o:18319405, Uchsdm:o:18708929, Uchsdm:o:51681337]"
2,"[{'record_id': 1, 'record_data': 'Uchsdm:o:68731001;68244063;68731009;68268059;35460835;18705931;68245549;35456853', 'record_len': 83, 'timestamp': '0:34:27'}, {'record_id': 2, 'record_data': 'Uchsdm:o:18732295', 'record_len': 20, 'timestamp': '0:34:46'}, {'record_id': 3, 'record_data': 'Uchsdm:o:51497637', 'record_len': 20, 'timestamp': '0:35:11'}, {'record_id': 4, 'record_data': 'Uchsdm:o:51681757', 'record_len': 20, 'timestamp': '0:35:50'}, {'record_id': 5, 'record_data': 'Uchsdm:o:18319359', 'record_len': 20, 'timestamp': '0:37:0'}]",5cmh,NaN,#2014-12-15T15:08:32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90ff826f0a001d00,ffff,,NaN,NaN,NaN,"[Uchsdm:o:68731001;68244063;68731009;68268059;35460835;18705931;68245549;35456853, Uchsdm:o:18732295, Uchsdm:o:51497637, Uchsdm:o:51681757, Uchsdm:o:18319359]"
3,"[{'record_id': 1, 'record_data': 'Uchsdm:o:68268457;18731063;68731007;18797477;51685175;51589113;18620635;68268575;68246011', 'record_len': 92, 'timestamp': '0:34:44'}, {'record_id': 2, 'record_data': 'Uchsdm:o:18732761', 'record_len': 20, 'timestamp': '0:35:20'}, {'record_id': 3, 'record_data': 'Uchsdm:o:51497723', 'record_len': 20, 'timestamp': '0:35:46'}, {'record_id': 4, 'record_data': 'Uchsdm:o:51497211', 'record_len': 20, 'timestamp': '0:37:2'}, {'record_id': 5, 'record_data': 'Uchsdm:o:51497517', 'record_len': 20, 'timestamp': '0:37:50'}, {'record_id': 6, 'record_data': 'Uchsdm:o:18728283', 'record_len': 20, 'timestamp': '0:38:11'}]",cwzn,NaN,#2014-12-15T15:07:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91ff826f08001b00,ffff,,NaN,NaN,NaN,"[Uchsdm:o:68268457;18731063;68731007;18797477;51685175;51589113;18620635;68268575;68246011, Uchsdm:o:18732761, Uchsdm:o:51497723, Uchsdm:o:51497211, Uchsdm:o:51497517, Uchsdm:o:18728283]"
4,"[{'record_id': 1, 'record_data': 'Uchsdm:o:68731001;68244063;68731009;68268059;35460835;18705931;68245549;35456853', 